In [1]:
using LinearAlgebra
using TensorKit
using KrylovKit
using JSON
using HDF5, JLD
cd("D:\\My Documents\\Code\\Julia_codes\\Tensor network\\IPEPS_TensorKit\\kagome\\SU2_PG")
#push!(LOAD_PATH, "D:\\My Documents\\Code\\Julia_codes\\Tensor network\\IPEPS_TensorKit\\kagome\\SU2_PG")
include("kagome_load_tensor.jl")
include("kagome_CTMRG.jl")
include("kagome_model.jl")
include("kagome_IPESS.jl")
include("mps_algorithms\\ITEBD_algorithms.jl")
include("mps_algorithms\\TransfOp_decomposition.jl")
include("mps_algorithms\\PUMPS_algorithms.jl")

D=3;
chi=5;
W=5;
N=6;
tol=1e-6;
EH_n=3;#number of entanglement spectrum per k point

Dtrun_init=200;
Dtrun_max=200;
trun_tol=1e-8;
group_size=Int(round((10^8)/(chi*chi*W*W*D)));

mpo_type="OO";#"O_O" or "OO"

pow=Int((N-2)/2);

J1=1;
J2=0;
J3=0;
Jchi=0;
Jtrip=0;


In [2]:

parameters=Dict([("J1", J1), ("J2", J2), ("J3", J3), ("Jchi", Jchi), ("Jtrip", Jtrip)]);

A_set,B_set,A1_set,A2_set, A_set_occu,B_set_occu,A1_set_occu,A2_set_occu, S_label, Sz_label, virtual_particle, Va, Vb=construct_tensor(D);

filenm="LS_D_"*string(D)*"_chi_40.json"
json_dict=read_json_state(filenm);

bond_tensor,triangle_tensor=construct_su2_PG_IPESS(json_dict,A_set,B_set,A1_set,A2_set, A_set_occu,B_set_occu,A1_set_occu,A2_set_occu, S_label, Sz_label, virtual_particle, Va, Vb);

PEPS_tensor=bond_tensor;
@tensor PEPS_tensor[:] := bond_tensor[-1,1,-5]*bond_tensor[4,3,-6]*bond_tensor[-4,2,-7]*triangle_tensor[1,3,2]*triangle_tensor[4,-2,-3];
A_unfused=PEPS_tensor;

U_phy=unitary(fuse(space(PEPS_tensor, 5) ⊗ space(PEPS_tensor, 6) ⊗ space(PEPS_tensor, 7)), space(PEPS_tensor, 5) ⊗ space(PEPS_tensor, 6) ⊗ space(PEPS_tensor, 7));
@tensor A_fused[:] :=PEPS_tensor[-1,-2,-3,-4,1,2,3]*U_phy[-5,1,2,3];


In [3]:
CTM=[];
U_L=[];
U_D=[];
U_R=[];
U_U=[];

try
    #load data
    jld_filenm="CTM_D"*string(D)*"_chi"*string(chi)*".jld";
    CTM_dict=load(jld_filenm)["CTM_dict"];
    U_L_dict=load(jld_filenm)["U_L_dict"];
    U_D_dict=load(jld_filenm)["U_D_dict"];
    U_R_dict=load(jld_filenm)["U_R_dict"];
    U_U_dict=load(jld_filenm)["U_U_dict"];
    CTM=deepcopy(CTM_dict)
    for cc=1:4
        CTM["Tset"][cc]=convert(TensorMap,CTM_dict["Tset"][cc]);
        CTM["Cset"][cc]=convert(TensorMap,CTM_dict["Cset"][cc]);
    end
    U_L=convert(TensorMap,U_L_dict);
    U_D=convert(TensorMap,U_D_dict);
    U_R=convert(TensorMap,U_R_dict);
    U_U=convert(TensorMap,U_U_dict);

    display("load CTM from saved data directly")
catch e

    display("No CTM found from saved data, now do CTMRG")


    init=Dict([("CTM", []), ("init_type", "PBC")]);
    conv_check="singular_value";
    @time CTM, AA_fused, U_L,U_D,U_R,U_U=CTMRG(A_fused,chi,conv_check,tol,init);
    
    @time E_up, E_down=evaluate_ob(parameters, U_phy, A_unfused, A_fused, AA_fused, U_L,U_D,U_R,U_U, CTM, "E_triangle");
    @time E_up_12, E_up_31, E_up_23, E_down_12, E_down_31, E_down_23=evaluate_ob(parameters, U_phy, A_unfused, A_fused, AA_fused, U_L,U_D,U_R,U_U, CTM, "E_bond");
    display((E_up+E_down)/3)

    
    CTM_dict=deepcopy(CTM)
    for cc=1:4
        CTM_dict["Tset"][cc]=convert(Dict,CTM_dict["Tset"][cc]);
        CTM_dict["Cset"][cc]=convert(Dict,CTM_dict["Cset"][cc]);
    end
    U_L_dict=convert(Dict,U_L);
    U_D_dict=convert(Dict,U_D);
    U_R_dict=convert(Dict,U_R);
    U_U_dict=convert(Dict,U_U);

    jld_filenm="CTM_D"*string(D)*"_chi"*string(chi)*".jld";
    save(jld_filenm, "CTM_dict",CTM_dict, "U_L_dict",U_L_dict,"U_D_dict",U_D_dict,"U_R_dict",U_R_dict,"U_U_dict",U_U_dict);



end


"load CTM from saved data directly"

In [4]:

O1=[];
O2=[];
Ag=[];
try
    #load data
    jld_filenm="itebd_ground_D"*string(D)*"_chi"*string(chi)*"_W"*string(W)*".jld";
    Ag_dict=load(jld_filenm)["Ag_dict"];
    O1_dict=load(jld_filenm)["O1_dict"];
    O2_dict=load(jld_filenm)["O2_dict"];
    Ag=convert(TensorMap,Ag_dict);
    O1=convert(TensorMap,O1_dict);
    O2=convert(TensorMap,O2_dict);
    display("load itebd ground state from saved data directly")
catch e

    display("No itebd ground state found from saved data, now do itebd")
    
    Tleft=CTM["Tset"][4];
    Tright=CTM["Tset"][2];
    @tensor O1[:]:=Tleft[-3,1,-1]*U_L[1,-2,-4];
    @tensor O2[:]:=Tright[-1,1,-3]*U_R[-4,-2,1];
    
    mps_virtual=SU₂Space(0=>1,1/2=>1,1=>1);mps_phy=space(O1,2);
    A_init=permute(TensorMap(randn, mps_virtual*mps_virtual', mps_phy),(1,2,3,),());

    #Ag,A_init=ITEBD_boundary_groundstate(O1,O2,W,A_init,"OO");
    Ag,A_init=ITEBD_boundary_groundstate(O1,O2,W,A_init,"O_O");

    #save itebd data
    O1_dict=convert(Dict,O1);
    O2_dict=convert(Dict,O2);
    Ag_dict=convert(Dict,Ag);
    jld_filenm="itebd_ground_D"*string(D)*"_chi"*string(chi)*"_W"*string(W)*".jld";
    save(jld_filenm, "O1_dict", O1_dict,"O2_dict", O2_dict,"Ag_dict", Ag_dict);

    #save initial CTM to compare with other codes
    matwrite("itebd_matfile.mat", Dict(
        "O1" => convert(Array,O1),
        "O2" => convert(Array,O2),
        "A_init" => convert(Array,A_init),
        "Ag" => convert(Array,Ag)
    ); compress = false)


end

"load itebd ground state from saved data directly"

In [5]:

space_AOA=fuse(space(Ag,1)'⊗space(O2,1)'⊗space(O1,1)⊗ space(Ag,1));
space_AA=fuse(space(Ag,1)'⊗ space(Ag,1));

AOA_sec=collect(sectors(space_AOA))
AA_sec=collect(sectors(space_AA))

@tensor OO[:]:=O1[-2,-3,-5,1]*O2[-1,1,-4,-6];
U_fuse_chichi=unitary(fuse(space(OO,1)⊗ space(OO,2)),space(OO,1)⊗ space(OO,2));
@tensor OO[:]:=U_fuse_chichi[-1,1,2]*OO[1,2,-2,3,4,-4]*U_fuse_chichi'[3,4,-3];
display(space(OO))

(Rep[SU₂](0=>5, 1/2=>4, 1=>4) ⊗ Rep[SU₂](0=>1, 1/2=>1) ⊗ Rep[SU₂](0=>5, 1/2=>4, 1=>4)' ⊗ Rep[SU₂](0=>1, 1/2=>1)') ← ProductSpace{GradedSpace{SU2Irrep, TensorKit.SortedVectorDict{SU2Irrep, Int64}}, 0}()

In [6]:
# for s in sectors(space_AOA)
#   display(s)
#   @show s, dim(space_AOA, s), dim(s)
# end


In [7]:
#normalize the MPO
euL_set,_,_,_,_=FLR_eig(Ag,OO,20,space_AOA,AOA_sec);
norm_coe=maximum(abs.(group_numbers(euL_set)));
OO=OO/norm_coe;
O1=O1/norm_coe;


In [8]:

check_complete_decomp=false;
if check_complete_decomp
    Dtrun=W*W*chi*chi;

    euL_set,evL_set,euR_set,evR_set,SPIN_eig_set=FLR_eig(Ag,OO,Dtrun,space_AOA,AOA_sec);
    display(size(euL_set))
    eul_set,evl_set,eur_set,evr_set,spin_eig_set=GLR_eig(Ag,Dtrun,space_AA,AA_sec);
    display(size(eul_set))
    S_set,U_set,Vh_set,SPIN_svd_set=FLR_svd(Ag,OO,pow,Dtrun,space_AOA,AOA_sec);
    display(size(S_set))
    s_set,u_set,vh_set,spin_svd_set=GLR_svd(Ag,pow,Dtrun,space_AA,AA_sec);
    display(size(s_set))


    euR_set,evL_set,evR_set,SPIN_eig_set=truncate_sectors(Dtrun,euR_set,evL_set,evR_set,SPIN_eig_set);
    display(euR_set)
    eur_set,evl_set,evr_set,spin_eig_set=truncate_sectors(Dtrun,eur_set,evl_set,evr_set,spin_eig_set);
    display(eur_set)
    S_set,U_set,Vh_set,SPIN_svd_set=truncate_sectors(Dtrun,S_set,U_set,Vh_set,SPIN_svd_set);
    display(S_set)
    s_set,u_set,vh_set,spin_svd_set=truncate_sectors(Dtrun,s_set,u_set,vh_set,spin_svd_set);
    display(s_set)




    display("check FLR_eig")
    @tensor M[:]:=Ag'[-1,-4,1]*OO[-2,2,-5,1]*Ag[-3,-6,2];
    M=permute(M,(1,2,3,),(4,5,6,));
    M0=M;
    for cc=1:pow-1
        M=M*M0;
    end
    M_temp=M*0;
    for cc=1:length(euR_set)
        euR=euR_set[cc];
        evR=evR_set[cc];
        evL=evL_set[cc];
        spin=SPIN_eig_set[cc]
        for cs=1:length(euR)
            @tensor coe=evR[cs][2,3,4,1]*evL[cs][1,2,3,4];
            M_temp=M_temp+euR[cs]^(pow)*evR[cs]*evL[cs]*(spin[cs]*2+1)/coe;
        end
    end
    display(norm(M-M_temp)/norm(M))

    euR_set_combined,evL_set_combined,evR_set_combined,SPIN_eig_set_combined=combine_singlespin_sector(euR_set,evL_set,evR_set,SPIN_eig_set,true)
    M_temp=M*0;
    for cc=1:length(euR_set)
        euR=euR_set_combined[cc];
        evR=evR_set_combined[cc];
        evL=evL_set_combined[cc];
        spin=SPIN_eig_set_combined[cc]
        if euR==[]
        else
            euR0=euR
            for cc=1:pow-1
                euR=euR*euR0;
            end

            M_temp=M_temp+evR*euR*evL*(spin*2+1);
        end
    end
    display(norm(M-M_temp)/norm(M))




    display("check GLR_eig")
    @tensor M[:]:=Ag'[-1,-3,1]*Ag[-2,-4,1];
    M=permute(M,(1,2,),(3,4,));
    M0=M;
    for cc=1:pow-1
        M=M*M0;
    end
    eur_set_combined,evl_set_combined,evr_set_combined,spin_eig_set_combined=combine_singlespin_sector(eur_set,evl_set,evr_set,spin_eig_set,true)

    M_temp=M*0;
    for cc=1:length(eul_set)
        eur=eur_set_combined[cc];
        evl=evl_set_combined[cc];
        evr=evr_set_combined[cc];
        spin=spin_eig_set_combined[cc]
        if eur==[]
        else
            eur0=eur;
            for cc=1:pow-1
                eur=eur*eur0;
            end
            M_temp=M_temp+evr*eur*evl*(spin*2+1);
        end
    end
    display(norm(M-M_temp)/norm(M))





    display("check FLR_svd")
    @tensor M[:]:=Ag'[-1,-4,1]*OO[-2,2,-5,1]*Ag[-3,-6,2];
    M=permute(M,(1,2,3,),(4,5,6,));
    M0=M;
    for cc=1:pow-1
        M=M*M0;
    end
    M_temp=M*0;
    for cc=1:length(S_set)
        S=S_set[cc];
        U=U_set[cc];
        Vh=Vh_set[cc];
        spin=SPIN_svd_set[cc]
        for cs=1:length(S)
            M_temp=M_temp+S[cs]*U[cs]*Vh[cs]*(spin[cs]*2+1);
        end
    end
    display(norm(M-M_temp)/norm(M))

    S_set_combined,Vh_set_combined,U_set_combined,SPIN_svd_set_combined=combine_singlespin_sector(S_set,Vh_set,U_set,SPIN_svd_set,false)
    M_temp=M*0;
    for cc=1:length(S_set)
        S=S_set_combined[cc];
        U=U_set_combined[cc];
        Vh=Vh_set_combined[cc];
        spin=SPIN_svd_set_combined[cc]
        if S==[]
        else
            M_temp=M_temp+U*S*Vh*(spin*2+1);
        end
    end
    display(norm(M-M_temp)/norm(M))



    display("check GLR_svd")
    @tensor M[:]:=Ag'[-1,-3,1]*Ag[-2,-4,1];
    M=permute(M,(1,2,),(3,4,));
    M0=M;
    for cc=1:pow-1
        M=M*M0;
    end
    M_temp=M*0;
    for cc=1:length(s_set)
        s=s_set[cc];
        u=u_set[cc];
        vh=vh_set[cc];
        spin=spin_svd_set[cc]
        for cs=1:length(s)
            M_temp=M_temp+s[cs]*u[cs]*vh[cs]*(spin[cs]*2+1);
        end
    end
    display(norm(M-M_temp)/norm(M))

    s_set_combined,vh_set_combined,u_set_combined,spin_svd_set_combined=combine_singlespin_sector(s_set,vh_set,u_set,spin_svd_set,false)


    M_temp=M*0;
    for cc=1:length(s_set)
        s=s_set_combined[cc];
        vh=vh_set_combined[cc];
        u=u_set_combined[cc];
        spin=spin_svd_set_combined[cc]
        if s==[]
        else
            M_temp=M_temp+u*s*vh*(spin*2+1);
        end
    end
    display(norm(M-M_temp)/norm(M))

end


In [9]:
euR_set,evL_set,evR_set,SPIN_eig_set=TransfOp_decom(Ag,OO,space_AOA,AOA_sec,pow,Dtrun_init,Dtrun_max,trun_tol,"eigenvalue_FLR");
# display(euR_set)

eur_set,evl_set,evr_set,spin_eig_set=TransfOp_decom(Ag,OO,space_AA,AA_sec,pow,Dtrun_init,Dtrun_max,trun_tol,"eigenvalue_GLR");
# display(eur_set)

"FLR_eig, DTrun=200, SU2 reduced to 94, minimal eigenvalue: 0.005102569466384027"

"GLR_eig, Dtrun=200, SU2 reduced to 13, minimal eigenvalue: 0.0012072930639182953"

"Dtrun exceeds matrix size"

(Any[ComplexF64[1.0000000000000062 + 0.0im, -0.1285411042588198 - 0.6403596952899754im, -0.12854110425881946 + 0.6403596952899755im, 0.06838125896513747 - 5.756357788026782e-18im, -0.06568417212048767 - 1.993476978253688e-17im], ComplexF64[0.5730937834312549 + 0.3831799476835856im, 0.573093783431255 - 0.38317994768358543im, -0.20630574949032854 + 0.38699355565278803im, -0.2063057494903283 - 0.3869935556527881im], ComplexF64[0.13457606824583213 - 3.7834420655007833e-17im, 0.06838125224346751 + 3.10240426989962e-5im, 0.0683812522434672 - 3.102404269893237e-5im, 0.034746123005571575 - 3.6089941275177256e-17im]], Any[Any[TensorMap(ProductSpace(Rep[SU₂](0=>1)) ← (Rep[SU₂](0=>1, 1/2=>2) ⊗ Rep[SU₂](0=>1, 1/2=>2)')):
* Data for fusiontree FusionTree{Irrep[SU₂]}((0,), 0, (false,), ()) ← FusionTree{Irrep[SU₂]}((1/2, 1/2), 0, (false, true), ()):
[:, :, 1] =
 -0.07261868893441695 - 0.628272652609874im  …  -4.5970172113385406e-17 - 8.465450562766823e-16im

[:, :, 2] =
 4.8572257327350623e-17 - 1.04

In [10]:

S_set,U_set,Vh_set,SPIN_svd_set=TransfOp_decom(Ag,OO,space_AOA,AOA_sec,pow,Dtrun_init,Dtrun_max,trun_tol,"svd_FLR");
# display(S_set)

s_set,u_set,vh_set,spin_svd_set=TransfOp_decom(Ag,OO,space_AA,AA_sec,pow,Dtrun_init,Dtrun_max,trun_tol,"svd_GLR");
# display(s_set)

"minimal singular value in this sector is quite small, skip checking number of converged values"

"minimal singular value in this sector is quite small, skip checking number of converged values"

"FLR_svd, DTrun=200, SU2 reduced to 91, minimal eigenvalue: 0.002240726462392007"

"GLR_svd, Dtrun=200, SU2 reduced to 13, minimal eigenvalue: 6.0356911778564504e-5"

"Dtrun exceeds matrix size"

Number of singular values obtained are not enough, use smaller tol
Number of singular values obtained are not enough, add noise, increase Krylov subspace and use smaller tol
Number of singular values obtained are not enough, add noise, increase Krylov subspace and use smaller tol


┌ Warning: Invariant subspace of dimension 11 (up to requested tolerance `tol = 1.0e-12`), which is smaller than the number of requested singular values (i.e. `howmany == 16`); setting `howmany = 11`.
└ @ KrylovKit C:\Users\Lenovo\.julia\packages\KrylovKit\YPiz7\src\eigsolve\svdsolve.jl:142
┌ Warning: Invariant subspace of dimension 11 (up to requested tolerance `tol = 1.0e-14`), which is smaller than the number of requested singular values (i.e. `howmany == 16`); setting `howmany = 11`.
└ @ KrylovKit C:\Users\Lenovo\.julia\packages\KrylovKit\YPiz7\src\eigsolve\svdsolve.jl:142
┌ Warning: Invariant subspace of dimension 2 (up to requested tolerance `tol = 1.0e-12`), which is smaller than the number of requested singular values (i.e. `howmany == 4`); setting `howmany = 2`.
└ @ KrylovKit C:\Users\Lenovo\.julia\packages\KrylovKit\YPiz7\src\eigsolve\svdsolve.jl:142
┌ Warning: number of values converged is not enough
└ @ Main d:\My Documents\Code\Julia_codes\Tensor network\IPEPS_TensorKit\ka

(Any[[1.3176594821247485, 0.4826343774428261, 0.3598356098545592, 0.004675996562811555, 0.0034308654699885376], [0.4840766885277672, 0.48407668802794923, 0.1888244939731826, 0.18882449372694707], [0.2749274959160688, 0.004675996649053501, 0.00467599660750583, 7.952985711679243e-5]], Any[Any[TensorMap((Rep[SU₂](0=>1, 1/2=>2) ⊗ Rep[SU₂](0=>1, 1/2=>2)') ← ProductSpace(Rep[SU₂](0=>1))):
* Data for fusiontree FusionTree{Irrep[SU₂]}((1/2, 1/2), 0, (false, true), ()) ← FusionTree{Irrep[SU₂]}((0,), 0, (false,), ()):
[:, :, 1] =
 -0.36951777702333555 + 8.330587133061554e-7im  …   -0.0390151889908791 + 4.664195250352903e-6im
 -0.03901518901151293 - 4.488279799621472e-6im     -0.14936630569667203 + 3.3673860953658544e-7im
* Data for fusiontree FusionTree{Irrep[SU₂]}((0, 0), 0, (false, true), ()) ← FusionTree{Irrep[SU₂]}((0,), 0, (false,), ()):
[:, :, 1] =
 -0.9154790817892544 + 2.0639002325299193e-6im
, TensorMap((Rep[SU₂](0=>1, 1/2=>2) ⊗ Rep[SU₂](0=>1, 1/2=>2)') ← ProductSpace(Rep[SU₂](0=>1))):


In [11]:
check_truncated_decomp_error=false;

if mpo_type=="O_O"
    OO_transform=true;
elseif mpo_type=="OO"
    OO_transform=false;
end

euR_set_combined,evL_set_combined,evR_set_combined,SPIN_eig_set_combined=combine_singlespin_sector(euR_set,evL_set,evR_set,SPIN_eig_set,true);
euR_set_grouped,evL_set_grouped,evR_set_grouped,SPIN_eig_set_grouped,DTrun_FLR_eig=group_singlespin_sector(group_size,euR_set_combined,evL_set_combined,evR_set_combined,SPIN_eig_set_combined,OO_transform,U_fuse_chichi)
display("group information:")
display(DTrun_FLR_eig)

eur_set_combined,evl_set_combined,evr_set_combined,spin_eig_set_combined=combine_singlespin_sector(eur_set,evl_set,evr_set,spin_eig_set,true)
eur_set_grouped,evl_set_grouped,evr_set_grouped,spin_eig_set_grouped,Dtrun_GLR_eig=group_singlespin_sector(group_size,eur_set_combined,evl_set_combined,evr_set_combined,spin_eig_set_combined,false,[])
display("group information:")
display(Dtrun_GLR_eig)


S_set_combined,Vh_set_combined,U_set_combined,SPIN_svd_set_combined=combine_singlespin_sector(S_set,Vh_set,U_set,SPIN_svd_set,false)
S_set_grouped,Vh_set_grouped,U_set_grouped,SPIN_svd_set_grouped,DTrun_FLR_svd=group_singlespin_sector(group_size,S_set_combined,Vh_set_combined,U_set_combined,SPIN_svd_set_combined,OO_transform,U_fuse_chichi)
display("group information:")
display(DTrun_FLR_svd)

s_set_combined,vh_set_combined,u_set_combined,spin_svd_set_combined=combine_singlespin_sector(s_set,vh_set,u_set,spin_svd_set,false)
s_set_grouped,vh_set_grouped,u_set_grouped,spin_svd_set_grouped,Dtrun_GLR_svd=group_singlespin_sector(group_size,s_set_combined,vh_set_combined,u_set_combined,spin_svd_set_combined,false,[])
display("group information:")
display(Dtrun_GLR_svd)



if check_truncated_decomp_error

    display("%%%%%%%%%%%%%%%%%%%%%%")
    display("check FLR_eig")
    @tensor M[:]:=Ag'[-1,-4,1]*OO[-2,2,-5,1]*Ag[-3,-6,2];
    M=permute(M,(1,2,3,),(4,5,6,));
    M0=M;
    for cc=1:pow-1
        M=M*M0;
    end

    M_temp=M*0;
    for cc=1:length(euR_set)
        euR=euR_set_combined[cc];
        evR=evR_set_combined[cc];
        evL=evL_set_combined[cc];
        spin=SPIN_eig_set_combined[cc]
        if euR==[]
        else
            euR0=euR
            for ccc=1:pow-1
                euR=euR*euR0;
            end
            M_temp=M_temp+evR*euR*evL*(spin*2+1);
        end
    end
    display(norm(M-M_temp)/norm(M))

    M_temp1=M_temp*0;
    for cc=1:length(euR_set)
        euRs=euR_set_grouped[cc];
        evRs=evR_set_grouped[cc];
        evLs=evL_set_grouped[cc];
        spin=SPIN_eig_set_grouped[cc]
        if euRs==[]
        else
            for cs=1:length(euRs)
                euR0=euRs[cs];
                euR=euR0;
                for ccc=1:pow-1
                    euR=euR*euR0;
                end
                @tensor evR[:]:=U_fuse_chichi[-2,1,2]*evRs[cs][-1,1,2,-3,-4];
                evR=permute(evR,(1,2,3,),(4,));
                @tensor evL[:]:=evLs[cs][-1,-2,1,2,-4]*U_fuse_chichi'[1,2,-3];
                evL=permute(evL,(1,),(2,3,4,));
                M_temp1=M_temp1+evR*euR*evL*(spin*2+1);
            end
        end
    end

    display(norm(M-M_temp1)/norm(M))



    display("%%%%%%%%%%%%%%%%%%%%%%")
    display("check GLR_eig")
    @tensor M[:]:=Ag'[-1,-3,1]*Ag[-2,-4,1];
    M=permute(M,(1,2,),(3,4,));
    M0=M;
    for cc=1:pow-1
        M=M*M0;
    end

    M_temp=M*0;
    for cc=1:length(eur_set)
        eur=eur_set_combined[cc];
        evl=evl_set_combined[cc];
        evr=evr_set_combined[cc];
        spin=spin_eig_set_combined[cc]
        if eur==[]
        else
            eur0=eur;
            for ccc=1:pow-1
                eur=eur*eur0;
            end
            M_temp=M_temp+evr*eur*evl*(spin*2+1);
        end
    end
    display(norm(M-M_temp)/norm(M))

    M_temp1=M_temp*0;
    for cc=1:length(eur_set)
        eurs=eur_set_grouped[cc];
        evrs=evr_set_grouped[cc];
        evls=evl_set_grouped[cc];
        spin=spin_eig_set_grouped[cc]
        if eurs==[]
        else
            for cs=1:length(eurs)
                eur0=eurs[cs];
                eur=eur0;
                for ccc=1:pow-1
                    eur=eur*eur0;
                end

                M_temp1=M_temp1+evrs[cs]*eur*evls[cs]*(spin*2+1);
            end
        end
    end
    display(norm(M-M_temp1)/norm(M))




    display("%%%%%%%%%%%%%%%%%%%%%%")
    display("check FLR_svd")
    @tensor M[:]:=Ag'[-1,-4,1]*OO[-2,2,-5,1]*Ag[-3,-6,2];
    M=permute(M,(1,2,3,),(4,5,6,));
    M0=M;
    for cc=1:pow-1
        M=M*M0;
    end
    M_temp=M*0;
    for cc=1:length(S_set)
        S=S_set[cc];
        U=U_set[cc];
        Vh=Vh_set[cc];
        spin=SPIN_svd_set[cc]
        for cs=1:length(S)
            M_temp=M_temp+S[cs]*U[cs]*Vh[cs]*(spin[cs]*2+1);
        end
    end
    display(norm(M-M_temp)/norm(M))

    M_temp=M*0;
    for cc=1:length(S_set)
        S=S_set_combined[cc];
        U=U_set_combined[cc];
        Vh=Vh_set_combined[cc];
        spin=SPIN_svd_set_combined[cc]
        if S==[]
        else
            M_temp=M_temp+U*S*Vh*(spin*2+1);
        end
    end
    display(norm(M-M_temp)/norm(M))

    M_temp1=M_temp*0;
    for cc=1:length(S_set)
        Ss=S_set_grouped[cc];
        Us=U_set_grouped[cc];
        Vhs=Vh_set_grouped[cc];
        spin=SPIN_svd_set_grouped[cc]
        if Ss==[]
        else
            for cs=1:length(Ss)
                S=Ss[cs];
                @tensor U[:]:=U_fuse_chichi[-2,1,2]*Us[cs][-1,1,2,-3,-4];
                U=permute(U,(1,2,3,),(4,));
                @tensor Vh[:]:=Vhs[cs][-1,-2,1,2,-4]*U_fuse_chichi'[1,2,-3];
                Vh=permute(Vh,(1,),(2,3,4,));
                M_temp1=M_temp1+U*S*Vh*(spin*2+1);
            end
        end
    end
    display(norm(M-M_temp1)/norm(M))



    display("%%%%%%%%%%%%%%%%%%%%%%")
    display("check GLR_svd")
    @tensor M[:]:=Ag'[-1,-3,1]*Ag[-2,-4,1];
    M=permute(M,(1,2,),(3,4,));
    M0=M;
    for cc=1:pow-1
        M=M*M0;
    end
    M_temp=M*0;
    for cc=1:length(s_set)
        s=s_set[cc];
        u=u_set[cc];
        vh=vh_set[cc];
        spin=spin_svd_set[cc]
        for cs=1:length(s)
            M_temp=M_temp+s[cs]*u[cs]*vh[cs]*(spin[cs]*2+1);
        end
    end
    display(norm(M-M_temp)/norm(M))

    M_temp=M*0;
    for cc=1:length(s_set)
        s=s_set_combined[cc];
        vh=vh_set_combined[cc];
        u=u_set_combined[cc];
        spin=spin_svd_set_combined[cc]
        if s==[]
        else
            M_temp=M_temp+u*s*vh*(spin*2+1);
        end
    end
    display(norm(M-M_temp)/norm(M))

    M_temp1=M_temp*0;
    for cc=1:length(s_set)
        ss=s_set_grouped[cc];
        us=u_set_grouped[cc];
        vhs=vh_set_grouped[cc];
        spin=spin_svd_set_grouped[cc]
        if ss==[]
        else
            for cs=1:length(ss)
                s=ss[cs];
                M_temp1=M_temp1+us[cs]*s*vhs[cs]*(spin*2+1);
            end
        end
    end

    display(norm(M-M_temp1)/norm(M))
end

"group information:"

5-element Vector{Any}:
 UnitRange{Int64}[1:27]
 UnitRange{Int64}[1:36]
 UnitRange{Int64}[1:23]
 UnitRange{Int64}[1:8]
 ComplexF64[]

"group information:"

3-element Vector{Any}:
 UnitRange{Int64}[1:5]
 UnitRange{Int64}[1:4]
 UnitRange{Int64}[1:4]

"group information:"

5-element Vector{Any}:
 UnitRange{Int64}[1:25]
 UnitRange{Int64}[1:32]
 UnitRange{Int64}[1:25]
 UnitRange{Int64}[1:9]
 Float64[]

"group information:"

3-element Vector{Any}:
 UnitRange{Int64}[1:5]
 UnitRange{Int64}[1:4]
 UnitRange{Int64}[1:4]

In [12]:
Dtrun_method="svds";
ES_sectors=[0,1/2,1,3/2,2,5/2];
kset=0:N-1;
#kset=0:0
Eset=[];
Trun_err=0;
DTrun=0;
print("calculate ES for N="*string(N));
display("kset="*string(kset));
pow=round((N-2)/2);



if Dtrun_method=="eigs"
    DTrun=length(group_numbers(SPIN_eig_set));
    display("DTrun="*string(DTrun));

    euRs=abs.(group_numbers(euR_set));
    Trun_err=(minimum(euRs)/maximum(euRs))^pow;


    euR_pow=deepcopy(euR_set_grouped);
    for ca=1:length(euR_pow)
        for cb=1:length(euR_pow[ca])
            euR_pow[ca][cb]=euR_pow[ca][cb]^Int(pow);
        end
    end

    kset,Eset=solve_ITEBD_excitation_TrunTransOp_iterative(Ag,O1,O2,OO,EH_n,N,kset,ES_sectors,pow,evR_set_grouped,euR_pow,evL_set_grouped,SPIN_eig_set_grouped,DTrun_FLR_eig,mpo_type)

elseif Dtrun_method=="svds"
    DTrun=length(group_numbers(SPIN_svd_set));
    display("DTrun="*string(DTrun));

    Ss=abs.(group_numbers(S_set));
    Trun_err=(minimum(Ss)/maximum(Ss));

    kset,Eset=solve_ITEBD_excitation_TrunTransOp_iterative(Ag,O1,O2,OO,EH_n,N,kset,ES_sectors,pow,U_set_grouped,S_set_grouped,Vh_set_grouped,SPIN_svd_set_grouped,DTrun_FLR_svd,mpo_type)
end

ES_filenm="ES_"*Dtrun_method*"_D"*string(D)*"_chi"*string(chi)*"_W"*string(W)*"_N"*string(N)*".mat";
matwrite(ES_filenm, Dict(
    "kset" => convert(Vector,kset),
    "ES_sectors" => ES_sectors,
    "Eset" => Eset,
    "Trun_err"=>Trun_err,
    "DTrun"=>DTrun
); compress = false)


"kset=0:5"

"DTrun=91"

"sector k=0, spin=0.0:"

5-element Vector{ComplexF64}:
   0.9489266550466101 - 0.5505849050840901im
   0.9447189847688621 + 0.5479755565400156im
  0.17041675382312713 - 0.08412249496829916im
   0.1704283754063375 + 0.08407879234172883im
 0.050426655347805445 - 2.5998111770348472e-6im

"sector k=0, spin=0.5:"

...

"sector k=5, spin=2.0:"

"MPS decomposition does not have this sector, skip it"

"sector k=5, spin=2.5:"

"MPS decomposition does not have this sector, skip it"

calculate ES for N=6 61.783766 seconds (155.30 M allocations: 8.454 GiB, 6.07% gc time, 99.25% compilation time)
  1.442155 seconds (3.58 M allocations: 346.631 MiB, 4.02% gc time, 49.69% compilation time)
  0.481193 seconds (1.61 M allocations: 182.269 MiB, 13.43% gc time, 14.02% compilation time)
  0.458270 seconds (1.51 M allocations: 168.177 MiB, 13.24% gc time, 14.03% compilation time)
  0.288776 seconds (906.93 k allocations: 94.315 MiB, 12.11% compilation time)
  0.717075 seconds (2.22 M allocations: 238.130 MiB, 8.10% gc time)
  0.369111 seconds (1.02 M allocations: 109.456 MiB, 15.02% gc time)
  0.280276 seconds (980.05 k allocations: 104.823 MiB)
  0.267547 seconds (875.44 k allocations: 92.635 MiB)
  0.729260 seconds (2.22 M allocations: 238.130 MiB, 7.55% gc time)
  0.350168 seconds (1.02 M allocations: 109.456 MiB, 14.81% gc time)
  0.287103 seconds (980.05 k allocations: 104.823 MiB)
  0.315091 seconds (875.44 k allocations: 92.635 MiB, 15.85% gc time)
  0.654954 seconds 